In [20]:
%matplotlib inline
# not sure if necessary

In [21]:
import finetune
import importlib
import torch

In [33]:
# run this cell when you change stuff in the finetune module, and don't want to restart the notebook's kernel.
importlib.reload(finetune)
print(finetune.structure_checkpoints())

None


In [23]:
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 1
finetune.FEATURE_EXTRACT = False
finetune.DATASET = 'StanfordCars'
finetune.print_hypers() # shows params
# Initialize the model for this run
model_ft = finetune.initialize_model(_verbose=False)

classifier input size: None
batch size: 8
num epochs: 1
feature extract: False


In [24]:
model, hist, state = finetune.finetune_model(model_ft)

  0%|          | 0/1018 [00:00<?, ?it/s]

Epoch 0/0
----------


  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 4.8950 Acc: 0.0433


100%|██████████| 503/503 [00:10<00:00, 49.97it/s]


val Loss: 3.8709 Acc: 0.1547

Training complete in 0m 54s
Best val Acc: 0.154726


In [26]:
file_path = finetune.FINETUNED_MODELS_PATH+'/resnet.pth'
finetune.save_model(state, file_path)

dict_keys(['name', 'epochs', 'model_state_dict', 'optimizer_state_dict'])


In [9]:
state = torch.load(finetune.FINETUNED_MODELS_PATH+'/resnet.pth')
new_model = finetune.get_model_architecture('resnet')
new_model.load_state_dict(state['model_state_dict'])

optimizer_state_dict = state['optimizer_state_dict']

In [10]:
finetune.NUM_EPOCHS = 10
finetune.FEATURE_EXTRACT = False
finetune.DATASET = 'StanfordCars'
new_model, hist2, optimizer = finetune.finetune_model(new_model,optimizer_state_dict)

  0%|          | 0/1018 [00:00<?, ?it/s]

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 1.2683 Acc: 0.6916


  0%|          | 0/1018 [00:00<?, ?it/s]

...

val Loss: 0.6605 Acc: 0.8100

Epoch 9/9
----------


  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 0.8734 Acc: 0.7806


100%|██████████| 503/503 [00:09<00:00, 51.48it/s]

val Loss: 0.6831 Acc: 0.8030

Training complete in 8m 30s
Best val Acc: 0.809950


In [11]:
hist2

[tensor(0.7433, device='cuda:0', dtype=torch.float64),
 tensor(0.7761, device='cuda:0', dtype=torch.float64),
 tensor(0.7719, device='cuda:0', dtype=torch.float64),
 tensor(0.7856, device='cuda:0', dtype=torch.float64),
 tensor(0.7811, device='cuda:0', dtype=torch.float64),
 tensor(0.7896, device='cuda:0', dtype=torch.float64),
 tensor(0.8040, device='cuda:0', dtype=torch.float64),
 tensor(0.7920, device='cuda:0', dtype=torch.float64),
 tensor(0.8100, device='cuda:0', dtype=torch.float64),
 tensor(0.8030, device='cuda:0', dtype=torch.float64)]

In [ ]:
finetune.NUM_CLASSES = 196
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 15
finetune.FEATURE_EXTRACT = True
finetune.print_hypers() # shows params
# Initialize the model for this run
model_ft = finetune.initialize_model()


# Print the model we just instantiated
print(model_ft)
finetune.finetune_model(model_ft)

In [ ]:
finetune.NUM_CLASSES = 196
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 30
finetune.FEATURE_EXTRACT = True
finetune.print_hypers() # shows params
# Initialize the model for this run
model_ft = finetune.initialize_model()


# Print the model we just instantiated
print(model_ft)
finetune.finetune_model(model_ft)